In [4]:
library(ggplot2)
suppressPackageStartupMessages(library(extraDistr))
suppressPackageStartupMessages(library(distr))
source("./simple.R")
source("./simple_utils.R")
set.seed(2025)


### Q1: efficacy of vaccines

In [ ]:
# 2
BetaMP = function(mean, precision){
  Beta(mean*precision, (1-mean)*precision)
}

ppl_fn = function() {
    mu_e = simulate(Unif(0,1))
    lam_e = simulate(Exp(rate = 0.01))
    mu_p = simulate(BetaMP(0.1, 15))
    lam_p = simulate(Exp(rate = 0.01))
    # Pfizer
    e_pfizer = simulate(BetaMP(mu_e, lam_e))
    p_pfizer = simulate(BetaMP(mu_p, lam_p))
    observe(realization = 162, distribution = Binom(prob=p_pfizer, size=18325)) # n_c
    observe(realization = 8, distribution = Binom(prob=p_pfizer*(1 - e_pfizer), size=18198)) # n_v
    # Moderna
    e_moderna = simulate(BetaMP(mu_e, lam_e))
    p_moderna = simulate(BetaMP(mu_p, lam_p))
    observe(realization = 185, distribution = Binom(prob=p_moderna, size=14073)) # n_c
    observe(realization = 11, distribution = Binom(prob=p_moderna*(1 - e_moderna), size=14134)) # n_v
    return (ifelse(e_moderna > e_pfizer, 1, 0))
}

In [16]:
# 3
vaccines = read.csv("./vaccines.csv")
vaccines$groupSizes = as.double(vaccines$groupSizes) # needed due to bug in Binom code

In [17]:
# 4
posterior(ppl_fn, 10000)

[1] 0.06367245